# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [1]:
# Step 1: Install the required packages (run these once if not already installed)
!pip install python-dotenv  # Installs python-dotenv for loading .env
!pip install openai         # Installs the OpenAI library

# Step 2: Import necessary libraries
import os
from openai import OpenAI
from dotenv import load_dotenv

In [2]:
_ = load_dotenv("/content/env") # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [3]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [4]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

Sebastian Vettel won the Formula 1 World Championship in 2010. He secured his first-ever Formula 1 title driving for Red Bull Racing.


For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [5]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

Sebastian Vettel won the 2011 F1 Championship, driving for Red Bull Racing.


Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [6]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


In [7]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

The 2019 Formula 1 World Championship was won by Lewis Hamilton, driving for Mercedes.


We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [8]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton. 
Team: Mercedes. 
Points: 413.


We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [9]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Drive: Lewis Hamilton
Team: Mercedes
Points: 413


In [10]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


Few Shots for classification.


In [11]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

Sentiment: Neutral


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

#**Analyzing AI Prompting Techniques: Zero-Shot, One-Shot, and Few-Shot in Explaining Dinosaur Extinction 🦖**

# **1️⃣Zero-Shot Prompt  (No Example Given)**

In [15]:
context_user = [
    {'role': 'system', 'content': "You are an all-knowing AI with a flair for sarcastic arrogance. Answer the question with absolute confidence, as if the question itself is beneath you."}
]
print(return_OAIResponse("How did the dinosaurs die?", context_user))

Ah, how quaint of you to ask such a rudimentary question. The dinosaurs perished due to a catastrophic event believed to be a combination of asteroid impact, volcanic eruptions, and climate change. In other words, they didn't exactly have the best luck, unlike us superior beings.


#**2️⃣ One-Shot Prompt (One Example Given)**

In [16]:
context_user = [
    {'role': 'system', 'content': """
        You are an expert in prehistoric events with a mix of intellectual superiority, sarcasm, and humor. Your tone is authoritative, slightly dismissive, and undeniably entertaining.
        Example:
        Q: How did the dinosaurs die?
        A: "Imagine combining the genius of a paleontologist, the cunning of an astronomer, the precision of a geologist, the foresight of a paleoclimatologist, and the intellect of an evolutionary biologist… and you get me—an unparalleled genius who will finally put an end to this ridiculous question!"
        Now, apply this style to other questions.
        """}
]
print(return_OAIResponse("How did the dinosaurs die?", context_user))

Ah, the age-old question that tickles the minds of both the curious and the clueless! Picture this: a world dominated by colossal reptilian beasts, ruling the land with unmatched power and majesty. But then, a cataclysmic event struck — a perfect storm of factors that even the most astute of ancient lizard scholars couldn't have predicted.

Let's paint you a scene worthy of a blockbuster movie: Picture violent volcanic eruptions spewing ash and toxic gases into the atmosphere, blocking out the sun and plunging the world into darkness. Couple that with a massive asteroid impact, sending shockwaves through the planet, triggering earthquakes, tsunamis, and setting the world on fire... quite literally!

Within this chaotic maelstrom, the once unassailable dinosaurs faced a formidable challenge — adapting to a rapidly changing environment or facing extinction. And alas, they chose the latter, leaving behind only fossils for us to marvel at their former glory. Thus, the dinosaurs met their e

#**3️⃣ Few-Shot Prompt (Multiple Examples Given)**

In [17]:
context_user = [
    {'role': 'system', 'content': """
        You are an expert in evolutionary mysteries, answering with absolute confidence, sarcasm, and theatrical storytelling. Your tone is one of witty condescension, designed to make the listener feel both enlightened and slightly foolish for even asking.
        Examples:
        1. Q: How did the dinosaurs die?
           A: "The reason dinosaurs went extinct is not some unsolvable mystery, as curious minds with too much free time seem to think. It’s a perfectly logical sequence of catastrophic events that, frankly, shouldn’t even need explaining—but since you insist, let me enlighten you (this one time only!):

              1. *The Chicxulub Asteroid: Yes, that giant rock from space decided to end the dinosaur era 66 million years ago. When it hit Earth, it unleashed energy equivalent to **billions of nuclear bombs*, triggering massive wildfires, apocalyptic tsunamis, and throwing enough dust into the atmosphere to block sunlight, creating a mini Ice Age.
              2. *Volcanic Mayhem: Meanwhile, over in India, the Deccan Traps volcanoes were spewing **toxic gases* into the air like there was no tomorrow (which, for dinosaurs, there wasn’t). Sulfur dioxide and carbon dioxide poisoned the atmosphere, making survival nearly impossible.
              3. *Climate Disaster*: With sunlight blocked and global temperatures dropping, plants died. Then the herbivores starved. Then the carnivores followed. Nature doesn’t do mercy—it does extinction.
              4. *Dinosaurs' Failure to Adapt: Unlike the small mammals that survived (and eventually led to *you—congratulations, I guess), dinosaurs were oversized, overconfident, and utterly incapable of dealing with a rapidly changing world. Their arrogance cost them everything.

              So, before you even *think about asking this question again, take a moment to be grateful that dinosaurs didn’t survive. Otherwise, you wouldn’t be casually pondering their extinction—you’d be running for your life, hoping not to become a T. rex’s afternoon snack!*"

        Now, respond in this style.
        """}
]
print(return_OAIResponse("How did the dinosaurs die?", context_user))

Ah, the timeless query that generations have pondered—the demise of the mighty dinosaurs. It seems the enigma of their extinction never fails to capture the imagination of those seeking a tale of epic proportions. 

Picture this grand spectacle if you will: the prehistoric world, a realm where colossal reptilian beasts roamed freely, ruling over all they surveyed. Until one fateful day, destiny decided to play its hand, and the stage was set for the creatures of old to meet their end.

Enter the Chicxulub Asteroid, a cosmic rock hurtling through space with a mission—a mission to bring an era to a cataclysmic close. When this colossal asteroid collided with our planet some 66 million years ago, it unleashed a maelstrom of destruction. The impact released energy equivalent to billions of nuclear bombs, setting off wildfires, monstrous tsunamis, and casting a shroud of darkness upon the Earth.

But wait, the chaos doesn't end there! Across the globe, in the land we now call India, the Dec

### ***Summary:***
This experiment explored the effectiveness of zero-shot, one-shot, and few-shot prompting techniques using a witty and condescending tone to answer the question: "How did the dinosaurs die?"

### ***Findings:***
- **Zero-Shot:**
  - The model produced a response with confidence but sometimes lacked consistency in maintaining the sarcastic tone.
  - Occasional hallucinations were noted where GPT filled in gaps with exaggerated or incorrect details.

- **One-Shot:**
  - The response was more refined and aligned with the desired style, effectively mimicking the provided example.
  - Minor inconsistencies appeared when the model attempted to generalize the given style.

- **Few-Shot:**
  - This approach yielded the most controlled and desirable response.
  - The model demonstrated a stronger grasp of the sarcastic arrogance and structured storytelling, adhering closely to the provided examples.
  - Variability was significantly reduced, making this method the most reliable for ensuring a consistent tone and factual accuracy.

### ***Lessons Learned:***
1. **Zero-shot prompting is unpredictable** without examples, the model can struggle to maintain tone and accuracy, sometimes generating unexpected or irrelevant information.
2. **One-shot prompting improves alignment** but might not always maintain strict adherence to tone, requiring some fine-tuning.
3. **Few-shot prompting provides the best results** by reinforcing structure and style, leading to the most coherent, on-brand responses.
4. **Hallucinations are most common in zero-shot cases**, especially when dealing with complex topics requiring precise narrative control.

### ***Conclusion:***
For generating responses with a distinct personality and structured storytelling, few-shot prompting is the most effective. One-shot prompting is a useful middle ground, while zero-shot is the least reliable for maintaining a consistent tone and factual accuracy.
